In [2]:
from http.server import HTTPServer, SimpleHTTPRequestHandler, BaseHTTPRequestHandler, HTTPStatus
import http.client

class MyHandler(BaseHTTPRequestHandler):
    def do_PUT(self):
        print("PUT")
        self.close_connection = True
        print(self.rfile.readline())
        self.send_response(200)
        self.end_headers()
    
    def do_GET(self):
        print("GET")
        self.close_connection = False
        
    
    def parse_request(self):
        print("GOT REQUEST")
        """Parse a request (internal).

        The request should be stored in self.raw_requestline; the results
        are in self.command, self.path, self.request_version and
        self.headers.

        Return True for success, False for failure; on failure, an
        error is sent back.

        """
        self.command = None  # set in case of error on the first line
        self.request_version = version = self.default_request_version
        self.close_connection = True
        requestline = str(self.raw_requestline, 'iso-8859-1')
        requestline = requestline.rstrip('\r\n')
        self.requestline = requestline
        words = requestline.split()
        if len(words) == 3:
            command, path, version = words
            if version[:5] != 'HTTP/':
                self.send_error(
                    HTTPStatus.BAD_REQUEST,
                    "Bad request version (%r)" % version)
                return False
            try:
                base_version_number = version.split('/', 1)[1]
                version_number = base_version_number.split(".")
                # RFC 2145 section 3.1 says there can be only one "." and
                #   - major and minor numbers MUST be treated as
                #      separate integers;
                #   - HTTP/2.4 is a lower version than HTTP/2.13, which in
                #      turn is lower than HTTP/12.3;
                #   - Leading zeros MUST be ignored by recipients.
                if len(version_number) != 2:
                    raise ValueError
                version_number = int(version_number[0]), int(version_number[1])
            except (ValueError, IndexError):
                self.send_error(
                    HTTPStatus.BAD_REQUEST,
                    "Bad request version (%r)" % version)
                return False
            if version_number >= (1, 1) and self.protocol_version >= "HTTP/1.1":
                self.close_connection = False
            if version_number >= (2, 0):
                self.send_error(
                    HTTPStatus.HTTP_VERSION_NOT_SUPPORTED,
                    "Invalid HTTP Version (%s)" % base_version_number)
                return False
        elif len(words) == 2:
            command, path = words
            self.close_connection = True
            if command != 'GET':
                self.send_error(
                    HTTPStatus.BAD_REQUEST,
                    "Bad HTTP/0.9 request type (%r)" % command)
                return False
        elif not words:
            return False
        else:
            self.send_error(
                HTTPStatus.BAD_REQUEST,
                "Bad request syntax (%r)" % requestline)
            return False
        self.command, self.path, self.request_version = command, path, version

        # Examine the headers and look for a Connection directive.
        try:
            self.headers = http.client.parse_headers(self.rfile,
                                                     _class=self.MessageClass)
        except http.client.LineTooLong:
            self.send_error(
                HTTPStatus.BAD_REQUEST,
                "Line too long")
            return False
        except http.client.HTTPException as err:
            self.send_error(
                HTTPStatus.REQUEST_HEADER_FIELDS_TOO_LARGE,
                "Too many headers",
                str(err)
            )
            return False

        conntype = self.headers.get('Connection', "")
        if conntype.lower() == 'close':
            self.close_connection = True
        elif (conntype.lower() == 'keep-alive' and
              self.protocol_version >= "HTTP/1.1"):
            self.close_connection = False
        # Examine the headers and look for an Expect directive
        expect = self.headers.get('Expect', "")
        if (expect.lower() == "100-continue" and
                self.protocol_version >= "HTTP/1.1" and
                self.request_version >= "HTTP/1.1"):
            if not self.handle_expect_100():
                return False
        return True
    
class MyServer(HTTPServer):
    def __init__(self):
        super().__init__(("localhost", 12389), MyHandler)

In [3]:
s = MyServer()

In [8]:
import threading
t = threading.Thread(None, target=s.serve_forever)
t.start()

GOT REQUEST
PUT
b'PBSBioreactor,path_to_template.ini,PBS_X_DEMO_X,2018-05-16T00:31:22Z,AgPV(RPM),0.000000,TempPV(C),22.285763,DOPV(%),0.298695,pHPV,0.000000\r\n'
GOT REQUEST
PUT


127.0.0.1 - - [15/May/2018 17:31:27] "PUT / HTTP/1.1" 200 -


b'PBSBioreactor,path_to_template.ini,PBS_X_DEMO_X,2018-05-16T00:31:27Z,AgPV(RPM),0.000000,TempPV(C),22.285023,DOPV(%),0.303658,pHPV,0.000000\r\n'
GOT REQUEST
PUT


127.0.0.1 - - [15/May/2018 17:31:28] "PUT / HTTP/1.1" 200 -


b'PBSBioreactor,path_to_template.ini,PBS_X_DEMO_X,2018-05-16T00:31:28Z,AgPV(RPM),0.000000,TempPV(C),22.284853,DOPV(%),0.304890,pHPV,0.000000\r\n'
GOT REQUEST
PUT


...

127.0.0.1 - - [15/May/2018 17:31:39] "PUT / HTTP/1.1" 200 -


b'PBSBioreactor,path_to_template.ini,PBS_X_DEMO_X,2018-05-16T00:31:39Z,AgPV(RPM),0.000000,TempPV(C),22.286266,DOPV(%),0.311451,pHPV,0.000000\r\n'
GOT REQUEST
PUT


127.0.0.1 - - [15/May/2018 17:31:40] "PUT / HTTP/1.1" 200 -


b'PBSBioreactor,path_to_template.ini,PBS_X_DEMO_X,2018-05-16T00:31:40Z,AgPV(RPM),0.000000,TempPV(C),22.286423,DOPV(%),0.311951,pHPV,0.000000\r\n'
GOT REQUEST
PUT


127.0.0.1 - - [15/May/2018 17:31:41] "PUT / HTTP/1.1" 200 -


In [9]:
s.shutdown()

b'PBSBioreactor,path_to_template.ini,PBS_X_DEMO_X,2018-05-16T00:31:41Z,AgPV(RPM),0.000000,TempPV(C),22.286562,DOPV(%),0.312664,pHPV,0.000000\r\n'


127.0.0.1 - - [15/May/2018 17:31:42] "PUT / HTTP/1.1" 200 -
